In [ ]:
import toml
import pygsheets
from tqdm.auto import tqdm
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import benchlingapi
import sqlalchemy
from sqlalchemy import select
import xmltodict

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api.benchling as bapi
import paulssonlab.cloning.registry as registry
import paulssonlab.api.geneious as g

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
bench_session = benchlingapi.Session(config["benchling"]["api_key"])
benchling_folder = bapi.get_project_root(bench_session, config["benchling"]["project"])

In [ ]:
reg = registry.Registry(gc, config["registry"]["folder"], benchling_folder)

# Config

In [ ]:
lib_parts = reg[("LIB", "parts")]
olib_oligos = reg[("oLIB", "oligos")]
plib_maps = reg[("pLIB", "maps")]
part_types = reg[("LIB", "parts", "Part types")]

# Test

In [ ]:
db_url = sqlalchemy.engine.URL.create(**config["geneious_test"])
engine = sqlalchemy.create_engine(db_url)
Session = sqlalchemy.orm.sessionmaker(bind=engine)
session = Session()

In [ ]:
db_url.render_as_string(False)

In [ ]:
conn = engine.connect()

## Folders

In [ ]:
stmt = select(g.Folder).where(g.Folder.visible == True)
res = session.execute(stmt)

In [ ]:
res.all()

In [ ]:
_[0]

In [ ]:
x = _

In [ ]:
x.children

# Old

In [ ]:
metadata = sqlalchemy.MetaData()
metadata.reflect(bind=engine)

In [ ]:
metadata.tables.keys()

In [ ]:
conn.execute(select(metadata.tables["next_table_id"])).all()

In [ ]:
for name, t in metadata.tables.items():
    print(name)
    q = select([t]).limit(20)
    rows = conn.execute(q).all()
    for row in rows:
        print(row)
    print()

In [ ]:
q = select([metadata.tables["annotated_document"]])
rows = conn.execute(q).all()

In [ ]:
docs = [xmltodict.parse(row["document_xml"], dict_constructor=dict) for row in rows]

In [ ]:
BAD_CLASSES = [
    "com.biomatters.geneious.publicapi.implementations.sequence.OligoSequenceDocument",
    "com.biomatters.geneious.publicapi.documents.OperationRecordDocument",
]
good_docs = [d for d in docs if d["document"]["@class"] not in BAD_CLASSES]

In [ ]:
set(d["document"]["@class"] for d in good_docs)

In [ ]:
d[30]

In [ ]:
x = xmltodict.parse(row["document_xml"], dict_constructor=dict)